#### STT번역TTS / 실시간번역TTS 통합

In [5]:
import os
import time
import threading
from dotenv import load_dotenv
import azure.cognitiveservices.speech as speechsdk
from azure.ai.translation.text import TextTranslationClient
from azure.core.credentials import AzureKeyCredential
from datetime import datetime

# 환경변수 로드
load_dotenv()

# Azure 설정
speech_api_key = os.getenv("SPEECH_API_KEY")
region = os.getenv("SPEECH_REGION")
translator_api_key = os.getenv("TRANSLATOR_API_KEY")
translator_region = os.getenv("TRANSLATOR_REGION")
translator_endpoint = os.getenv("TRANSLATOR_ENDPOINT")

# ==================== 언어 설정 ====================
INPUT_LANGUAGE = "영어"      # STT 입력 언어
OUTPUT_LANGUAGE = "한국어"    # 번역 출력 + TTS 출력 언어

# 언어 매핑 테이블
LANGUAGE_MAPPING = {
    "한국어": ("ko-KR", "ko", "ko-KR-SunHiNeural"),
    "영어": ("en-US", "en", "en-US-AvaMultilingualNeural"),
    "중국어(간체)": ("zh-CN", "zh-Hans", "zh-CN-XiaoxiaoNeural"),
    "일본어": ("ja-JP", "ja", "ja-JP-NanamiNeural"),
    "독일어": ("de-DE", "de", "de-DE-KatjaNeural"),
    "프랑스어": ("fr-FR", "fr", "fr-FR-DeniseNeural"),
    "스페인어": ("es-ES", "es", "es-ES-ElviraNeural"),
    "이탈리아어": ("it-IT", "it", "it-IT-IsabellaNeural"),
    "러시아어": ("ru-RU", "ru", "ru-RU-DariyaNeural"),
    "포르투갈어": ("pt-BR", "pt", "pt-BR-FranciscaNeural"),
}

INPUT_STT_CODE, INPUT_TRANSLATE_CODE, INPUT_TTS_VOICE = LANGUAGE_MAPPING[INPUT_LANGUAGE]
OUTPUT_STT_CODE, OUTPUT_TRANSLATE_CODE, OUTPUT_TTS_VOICE = LANGUAGE_MAPPING[OUTPUT_LANGUAGE]

print(f"🎤 입력 언어: {INPUT_LANGUAGE} ({INPUT_STT_CODE})")
print(f"🔄 출력 언어: {OUTPUT_LANGUAGE} ({OUTPUT_TRANSLATE_CODE})")
print("=" * 50)

# ==================== 방법 1: STT → 번역 → TTS (3단계 파이프라인) ====================
class CustomTranslationPipeline:
    def __init__(self):
        # STT 설정
        self.speech_config = speechsdk.SpeechConfig(subscription=speech_api_key, region=region)
        self.speech_config.speech_recognition_language = INPUT_STT_CODE
        
        # 번역기 설정
        self.translator_credential = AzureKeyCredential(translator_api_key)
        self.translator = TextTranslationClient(
            credential=self.translator_credential,
            endpoint=translator_endpoint,
            region=translator_region
        )
        
        # TTS 설정
        self.tts_config = speechsdk.SpeechConfig(subscription=speech_api_key, region=region)
        self.tts_config.speech_synthesis_voice_name = OUTPUT_TTS_VOICE
        
        print("✅ STT → 번역 → TTS 파이프라인 초기화 완료")
    
    def translate_text(self, text, target_language=OUTPUT_TRANSLATE_CODE):
        """텍스트 번역"""
        try:
            response = self.translator.translate(
                body=[{"text": text}],
                to_language=[target_language]
            )
            
            if response and len(response) > 0:
                translation_result = response[0]
                translated_text = translation_result['translations'][0]['text']
                detected_language = translation_result.get('detectedLanguage', {}).get('language')
                
                return {
                    "original_text": text,
                    "translated_text": translated_text,
                    "detected_language": detected_language
                }
            else:
                return {"original_text": text, "translated_text": None, "error": "Empty response"}
        
        except Exception as e:
            print(f"❌ 번역 오류: {e}")
            return {"original_text": text, "translated_text": None, "error": str(e)}
    
    def speak_text(self, text):
        """TTS 음성 출력"""
        if not text.strip():
            return None
        
        try:
            audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
            synthesizer = speechsdk.SpeechSynthesizer(self.tts_config, audio_config)
            result = synthesizer.speak_text_async(text).get()
            
            if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
                print(f"🔊 [{OUTPUT_LANGUAGE}] TTS 출력 완료")
                return True
            else:
                print(f"❌ TTS 실패: {result.cancellation_details.reason}")
                return False
        except Exception as e:
            print(f"❌ TTS 오류: {e}")
            return None
    
    def continuous_recognition(self):
        """실시간 STT → 번역 → TTS"""
        print(f"🎤 [{INPUT_LANGUAGE} → {OUTPUT_LANGUAGE}] STT → 번역 → TTS 시작")
        print("말씀하시면 자동으로 번역되고 음성 출력됩니다.")
        print("종료하려면 'q' 또는 'quit'을 입력하세요.")
        print("-" * 50)
        
        audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
        speech_recognizer = speechsdk.SpeechRecognizer(self.speech_config, audio_config)
        
        done = False
        
        def recognized_handler(evt):
            if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech and evt.result.text.strip():
                original_text = evt.result.text.strip()
                print(f"\n✅ STT 인식: {original_text}")
                
                # 번역
                result = self.translate_text(original_text)
                if result.get("translated_text"):
                    translated = result["translated_text"]
                    detected = result.get("detected_language", "unknown")
                    print(f"🔄 번역 결과: {translated} (감지언어: {detected})")
                    
                    # TTS
                    self.speak_text(translated)
                else:
                    print("❌ 번역 실패")
                
                print("-" * 30)
        
        def recognizing_handler(evt):
            if evt.result.reason == speechsdk.ResultReason.RecognizingSpeech and evt.result.text.strip():
                print(f"🎤 인식중: {evt.result.text.strip()}", end='\r')
        
        def stop_handler(evt):
            nonlocal done
            done = True
        
        # 이벤트 핸들러 연결
        speech_recognizer.recognized.connect(recognized_handler)
        speech_recognizer.recognizing.connect(recognizing_handler)
        speech_recognizer.session_stopped.connect(stop_handler)
        speech_recognizer.canceled.connect(stop_handler)
        
        # 연속 인식 시작
        speech_recognizer.start_continuous_recognition()
        
        # 사용자 입력 대기
        def wait_for_user_input():
            nonlocal done
            while not done:
                try:
                    user_input = input().strip().lower()
                    if user_input in ['q', 'quit', 'exit', '종료']:
                        print("👋 사용자가 종료를 요청했습니다.")
                        done = True
                        break
                except (EOFError, KeyboardInterrupt):
                    print("\n👋 키보드 인터럽트로 종료합니다.")
                    done = True
                    break
        
        input_thread = threading.Thread(target=wait_for_user_input, daemon=True)
        input_thread.start()
        
        try:
            while not done:
                time.sleep(0.5)
        except KeyboardInterrupt:
            print("\n👋 키보드 인터럽트로 종료합니다.")
            done = True
        finally:
            print("🛑 STT → 번역 → TTS 중지...")
            speech_recognizer.stop_continuous_recognition()

# ==================== 방법 2: 실시간 번역 + TTS ====================
class RealtimeTranslation:
    def __init__(self):
        # 실시간 번역 설정
        self.translation_config = speechsdk.translation.SpeechTranslationConfig(
            subscription=speech_api_key, 
            region=region
        )
        self.translation_config.speech_recognition_language = INPUT_STT_CODE
        self.translation_config.add_target_language(OUTPUT_TRANSLATE_CODE)
        
        # TTS 설정
        self.tts_config = speechsdk.SpeechConfig(subscription=speech_api_key, region=region)
        self.tts_config.speech_synthesis_voice_name = OUTPUT_TTS_VOICE
        
        print("✅ 실시간 번역 + TTS 초기화 완료")
    
    def speak_text(self, text):
        """TTS 음성 출력"""
        try:
            audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
            synthesizer = speechsdk.SpeechSynthesizer(self.tts_config, audio_config)
            
            result = synthesizer.speak_text_async(text).get()
            
            if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
                print(f"🔊 [{OUTPUT_LANGUAGE}] TTS 출력 완료")
                return True
            else:
                print(f"❌ TTS 실패: {result.cancellation_details.reason}")
                return False
        except Exception as e:
            print(f"❌ TTS 오류: {e}")
            return False
    
    def translate_once(self):
        """한 번 번역"""
        print(f"🎤 [{INPUT_LANGUAGE} → {OUTPUT_LANGUAGE}] 한 번 번역")
        print("말씀하세요...")
        
        audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
        translation_recognizer = speechsdk.translation.TranslationRecognizer(
            translation_config=self.translation_config, 
            audio_config=audio_config
        )
        
        result = translation_recognizer.recognize_once_async().get()
        
        if result.reason == speechsdk.ResultReason.TranslatedSpeech:
            original_text = result.text
            translated_text = result.translations.get(OUTPUT_TRANSLATE_CODE, "")
            
            print(f"✅ 원본 ({INPUT_LANGUAGE}): {original_text}")
            print(f"🔄 번역 ({OUTPUT_LANGUAGE}): {translated_text}")
            
            if translated_text:
                self.speak_text(translated_text)
        
        elif result.reason == speechsdk.ResultReason.NoMatch:
            print("❌ 음성을 인식할 수 없습니다.")
        elif result.reason == speechsdk.ResultReason.Canceled:
            cancellation_details = result.cancellation_details
            print(f"❌ 인식 취소됨: {cancellation_details.reason}")
    
    def continuous_translation(self):
        """연속 실시간 번역"""
        print(f"🎤 [{INPUT_LANGUAGE} → {OUTPUT_LANGUAGE}] 실시간 연속 번역 시작")
        print("말씀하시면 자동으로 번역되고 음성 출력됩니다.")
        print("종료하려면 'q' 또는 'quit'을 입력하세요.")
        print("-" * 50)
        
        audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
        translation_recognizer = speechsdk.translation.TranslationRecognizer(
            translation_config=self.translation_config,
            audio_config=audio_config
        )
        
        done = False
        
        def recognized_handler(evt):
            if evt.result.reason == speechsdk.ResultReason.TranslatedSpeech:
                original_text = evt.result.text
                translated_text = evt.result.translations.get(OUTPUT_TRANSLATE_CODE, "")
                
                if original_text and translated_text:
                    print(f"\n✅ 실시간 번역: {original_text}")
                    print(f"🔄 번역 결과: {translated_text}")
                    
                    self.speak_text(translated_text)
                    print("-" * 30)
        
        def recognizing_handler(evt):
            if evt.result.reason == speechsdk.ResultReason.TranslatingSpeech:
                original_text = evt.result.text
                if original_text:
                    print(f"🎤 인식중: {original_text}", end='\r')
        
        def stopped_handler(evt):
            nonlocal done
            done = True
        
        # 이벤트 핸들러 연결
        translation_recognizer.recognized.connect(recognized_handler)
        translation_recognizer.recognizing.connect(recognizing_handler)
        translation_recognizer.canceled.connect(stopped_handler)
        translation_recognizer.session_stopped.connect(stopped_handler)
        
        # 연속 번역 시작
        translation_recognizer.start_continuous_recognition_async()
        
        # 사용자 입력 대기
        def wait_for_user_input():
            nonlocal done
            while not done:
                try:
                    user_input = input().strip().lower()
                    if user_input in ['q', 'quit', 'exit', '종료']:
                        print("👋 사용자가 종료를 요청했습니다.")
                        done = True
                        break
                except (EOFError, KeyboardInterrupt):
                    print("\n👋 키보드 인터럽트로 종료합니다.")
                    done = True
                    break
        
        input_thread = threading.Thread(target=wait_for_user_input, daemon=True)
        input_thread.start()
        
        try:
            while not done:
                time.sleep(0.5)
        except KeyboardInterrupt:
            print("\n👋 키보드 인터럽트로 종료합니다.")
            done = True
        finally:
            print("🛑 실시간 번역 + TTS 중지...")
            translation_recognizer.stop_continuous_recognition_async()

# ==================== 메인 통합 시스템 ====================
def main():
    print("🎯 Azure 실시간 번역 시스템")
    print("=" * 50)
    print("📋 번역 방식 선택:")
    print("1. STT → 번역 → TTS (연속 처리)")
    print("   ✅ 100개 이상 언어 지원")
    print("   ✅ 완전한 제어와 커스터마이징")
    print("   ✅ 화자 구분 확장 가능")
    print("")
    print("2. 실시간 번역 + TTS (연속)")
    print("   ✅ 낮은 지연시간")
    print("   ✅ 실시간 부분 결과")
    print("   ⚠️ 약 60개 언어 제한")
    print("")
    print("0. 종료")
    print("=" * 50)
    
    # 초기화는 사용자 선택 후에 수행
    custom_pipeline = None
    realtime_translation = None
    
    while True:
        choice = input("\n번역 방식을 선택하세요 > ").strip()
        
        if choice == "1":
            # STT → 번역 → TTS 방식 (연속)
            if custom_pipeline is None:
                custom_pipeline = CustomTranslationPipeline()
            
            custom_pipeline.continuous_recognition()
        
        elif choice == "2":
            # 실시간 번역 방식 (연속)
            if realtime_translation is None:
                realtime_translation = RealtimeTranslation()
            
            realtime_translation.continuous_translation()
        
        elif choice == "0":
            print("👋 프로그램을 종료합니다.")
            break
            
        else:
            print("❌ 잘못된 선택입니다.")

if __name__ == "__main__":
    main()


🎤 입력 언어: 영어 (en-US)
🔄 출력 언어: 한국어 (ko)
🎯 Azure 실시간 번역 시스템
📋 번역 방식 선택:
1. STT → 번역 → TTS (연속 처리)
   ✅ 100개 이상 언어 지원
   ✅ 완전한 제어와 커스터마이징
   ✅ 화자 구분 확장 가능

2. 실시간 번역 + TTS (연속)
   ✅ 낮은 지연시간
   ✅ 실시간 부분 결과
   ⚠️ 약 60개 언어 제한

0. 종료
✅ STT → 번역 → TTS 파이프라인 초기화 완료
🎤 [영어 → 한국어] STT → 번역 → TTS 시작
말씀하시면 자동으로 번역되고 음성 출력됩니다.
종료하려면 'q' 또는 'quit'을 입력하세요.
--------------------------------------------------
🎤 인식중: it is almost a three mile loop but it may just be where you spot a celebrity or more typically shirtless fitness gurus instagram models and plenty of dogs you'll have views of the griffith observatory hollywood sign downtown los angeles and on the clear day even the pacific ocean as you can probably see the best time to do this hike is at sunset it's a little less hot a little less crowded and has such magical views it is in one of my top five favorite places to
✅ STT 인식: It is almost a three mile loop, but it may just be where you spot a celebrity or more typically, shirtless fitness gurus, In